In [4]:
import time 
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_binary
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

browser.get('https://stockmages.netlify.app')

### 1. Srolling to reach the bottom

In [5]:
for i in tqdm(range(0,5000000,1000)):
    
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

  1%|          | 34/5000 [00:03<08:52,  9.32it/s]


KeyboardInterrupt: 

### 2. Scraping the Page Course

In [6]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

### 3. Checking number of images that has to be scraped

In [7]:
len(soup.find_all('div', class_ = 'container'))

196

### 4. Scraping the Image Details

In [8]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):

    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9])
    
    data.append([img_link, tags, likes, comments])

100%|██████████| 196/196 [00:00<00:00, 17802.88it/s]


### 5. Saving data in a DataFrame

In [9]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


### 6. Checking Null Values

In [10]:
df.isnull().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

### 7. Checking Duplicate Values

In [11]:
df['img_link'].describe()

count                                                   196
unique                                                  195
top       https://cdn.pixabay.com/photo/2022/04/10/19/28...
freq                                                      2
Name: img_link, dtype: object

### 8. Removing Duplicate Values

In [12]:
df = df.drop_duplicates()

### 9. Saving the DataFrame as CSV

In [13]:
df.to_csv('images.csv', index = False)